Limpieza para los datos 2.

In [155]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

Funciones

In [156]:
def separar_df(df):
    filtro = df['torneo'].str.contains('Men|ATP', regex=True, case = False)
    masc = df[filtro]
    return masc

def asignar_año(fecha_str):
    # Limpiar y convertir a día y mes (sin año)
    dia_mes = fecha_str.strip('/')  # Elimina '/' final
    dia, mes = map(int, dia_mes.split('/'))
    
    # Asignar 2024 si es antes de 11 de junio, 2025 en caso contrario
    if (mes < 6) or (mes == 6 and dia <= 11):
        return f"{fecha_str}2025"
    else:
        return f"{fecha_str}2024"

def df_jugador(jugador, masc):

    filtro_jugador = (masc['jugadorA'] == jugador) | (masc['jugadorB'] == jugador)
    df_jugador = masc[filtro_jugador].copy()

    df_jugador_A = df_jugador[df_jugador['jugadorA'] == jugador].copy()
    df_jugador_B = df_jugador[df_jugador['jugadorB'] == jugador].copy()

    columnas_A = [col for col in df_jugador.columns if col.endswith('A')]
    columnas_B = [col for col in df_jugador.columns if col.endswith('B')]
    mapeo_columnas = {**{col_a: col_b for col_a, col_b in zip(columnas_A, columnas_B)},
                    **{col_b: col_a for col_a, col_b in zip(columnas_A, columnas_B)}}

    df_jugador_B_invertido = df_jugador_B.rename(columns=mapeo_columnas)

    df_final = pd.concat([df_jugador_A, df_jugador_B_invertido], ignore_index=True)

    if 'fecha' in df_final.columns:
        df_final = df_final.sort_values('fecha', ascending=False)

    return df_final

Lectura

In [157]:
masc1 = pd.read_csv(".\\data\\data2\\acapulco_adelaide_masc_2025-06-12.csv")
masc2 = pd.read_csv(".\\data\\data2\\adelaide2-asiangames_masc_2025-06-12.csv")
masc3 = pd.read_csv(".\\data\\data2\\astana-bolzano_masc_2025-06-12.csv")
masc4 = pd.read_csv(".\\data\\data2\\bordeaux-frenchopen_masc_2025-06-12.csv")

dfs_masc = [masc1, masc2, masc3, masc4]
masc = pd.concat(dfs_masc, ignore_index=True)

print(len(masc))
masc.head()

2589


,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,dobleFaltasA,...,porcentajeJuegosGanadosAlSaqueB,JuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,JuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,PuntosGanadosRedA,PuntosGanadosRedB
0,acapulco,Machac T.,Davidovich Fokina A.,2,0,02.03. 04:10,no,11.0,4.0,NaN,...,80%,(8/10),20%,(2/10),0%,(0/10),(12/20),(8/20),NaN,NaN
1,acapulco,Shapovalov D.,Davidovich Fokina A.,0,2,01.03. 04:40,no,2.0,0.0,NaN,...,67%,(8/12),33%,(4/12),33%,(4/12),(12/24),(12/24),NaN,NaN
2,acapulco,Machac T.,Nakashima B.,2,1,01.03. 02:05,no,4.0,7.0,NaN,...,79%,(11/14),21%,(3/14),23%,(3/13),(13/27),(14/27),NaN,NaN
3,acapulco,Davidovich Fokina A.,Pacheco Mendez R.,2,0,28.02. 06:05,no,1.0,3.0,NaN,...,38%,(3/8),63%,(5/8),13%,(1/8),(12/16),(4/16),NaN,NaN
4,acapulco,Shapovalov D.,Giron M.,2,1,28.02. 03:35,no,3.0,2.0,NaN,...,62%,(8/13),38%,(5/13),21%,(3/14),(16/27),(11/27),NaN,NaN


In [158]:
masc['setsA'].unique()

array(['2', '0', '1', 'Ruud C.', 'Paul T.', 'Korda S.', '3',
       'Struff J-L.', 'Rosset M.', 'Isroilov S.', 'Safiullin R.',
       'Grosjean S.', 'Woodruff C.', 'Nargiso D.', 'Martinez P.',
       'Mronz A.', 'Thompson J.', 'Nishioka Y.', 'Auger-Aliassime F.',
       'Fils A.', 'Gaston H.'], dtype=object)

In [159]:
masc = masc[(masc['setsA'] != 'Fils A.') & (masc['setsA'] != 'Gaston H.') & (masc['setsA'] != 'Djokovic N.') & (masc['setsA'] != 'De Minaur A.')] #elimino 4 datos mal extraídos
valores_erroneos = ['Fils A.', 'Gaston H.', 'Djokovic N.', 'De Minaur A.', 'Ruud C.', 'Paul T.', 'Korda S.', 'Struff J-L.', 'Rosset M.', 'Isroilov S.', 'Safiullin R.', 'Grosjean S.', 'Woodruff C.', 'Nargiso D.', 'Martinez P.', 'Mronz A.', 'Thompson J.', 'Nishioka Y.', 'Auger-Aliassime F.']
masc = masc[~masc['setsA'].isin(valores_erroneos)]


masc['setsA'] = masc['setsA'].astype(int)
masc['setsB'] = masc['setsB'].astype(int)

filtroGrandSlam = (masc['torneo'] == 'australian-open') | (masc['torneo'] == 'us-open') | (masc['torneo'] == 'wimbledon') | (masc['torneo'] == 'french-open')
filtro3sets = (masc['setsA'] == 3) | (masc['setsB'] == 3)
filtro2sets = (masc['setsA'] == 2) | (masc['setsB'] == 2)
filtroGrandSlamGanaA = masc['setsA'] == 3
filtroGrandSlamGanaB = masc['setsB'] == 3
filtroNoGrandSlamGanaA = masc['setsA'] == 2
filtroNoGrandSlamGanaB = masc['setsB'] == 2

# Condición 1: es Grand Slam y no se llegó a 3 sets (partido incompleto/retirado)
cond1 = filtroGrandSlam & (~filtro3sets)
masc.loc[cond1, 'retirado'] = 'si'

# Condición 2: no es Grand Slam y no se llegó a 2 sets (partido incompleto/retirado)
cond2 = (~filtroGrandSlam) & (~filtro2sets)
masc.loc[cond2, 'retirado'] = 'si'

#Creo la variable superficie
torneos_superficies = [
    ('acapulco', 'Hard'),('adelaide', 'Hard'),('adelaide-2', 'Hard'),('almaty', 'Clay'),('amersfoort', 'Clay'),('amsterdam', 'Clay'),('antalya', 'Grass'),('antwerp', 'Hard'),('antwerp-2', 'Hard'),('asian-games', 'Hard'),
    ('astana', 'Hard'),('athens', 'Clay'),('atlanta', 'Hard'),('atp-cup', 'Hard'),('auckland', 'Hard'),('australian-open', 'Hard'),('bangkok', 'Hard'),('banja-luka', 'Clay'),('barcelona', 'Clay'),('basel', 'Hard'),
    ('bastad', 'Clay'),('beijing', 'Hard'),('belgrade', 'Clay'),('belgrade-2', 'Clay'),('berlin', 'Grass'),('bermuda', 'Hard'),('birmingham', 'Grass'),('bogota', 'Clay'),('bologna', 'Clay'),('bolzano', 'Hard'),
    ('bordeaux', 'Clay'),('boston', 'Hard'),('brasilia', 'Clay'),('brighton', 'Hard'),('brisbane', 'Hard'),('bucharest', 'Clay'),('budapest', 'Clay'),('buenos-aires', 'Clay'),('buzios', 'Clay'),('cagliari', 'Clay'),
    ('casablanca', 'Clay'),('chengdu', 'Hard'),('chennai', 'Hard'),('chicago', 'Hard'),('cincinnati', 'Hard'),('cologne', 'Hard'),('cologne-2', 'Hard'),('copenhagen', 'Hard'),('cordoba', 'Clay'),('costa-do-sauipe', 'Clay'),
    ('dallas', 'Hard'),('delray-beach', 'Hard'),('doha', 'Hard'),('dubai', 'Hard'),('dusseldorf', 'Clay'),('eastbourne', 'Grass'),('essen', 'Hard'),('estoril', 'Clay'),('florence', 'Hard'),('french-open', 'Clay'),
]

superficie_dict = dict(torneos_superficies)
masc['superficie'] = masc['torneo'].map(superficie_dict)


masc[['dia', 'hora']] = masc['fecha'].str.split(' ', expand=True) #pongo la fecha como de normal (13/01/2025)
masc['dia'] = masc['dia'].str.replace('.', '/', regex=False)
masc['fecha'] = masc['dia'].apply(asignar_año)
masc = masc.drop(columns = ['dia', 'cuotaA', 'cuotaB'])
masc['fecha'] = pd.to_datetime(masc['fecha'], format='%d/%m/%Y')
masc['sets'] = masc['setsA'] + masc['setsB']

#Me quedo con las cuotas buenas
masc = masc.rename(columns={
    'cuotaA_2': 'cuotaA',
    'cuotaB_2': 'cuotaB'
})

#Le quito el símbolo del porcentaje a las columnas que lo tienen
cols_porcentaje = [col for col in masc.columns if 'porcentaje' in col.lower()]
for col in cols_porcentaje:
    masc[col] = masc[col].astype(str).str.replace('%', '', regex=False)
    masc[col] = pd.to_numeric(masc[col], errors='coerce')

#Le quito los paréntesis a las columnas que lo tienen
cols_con_par = [col for col in masc.columns if masc[col].astype(str).str.contains(r'\(\d+/\d+\)').any()]
for col in cols_con_par:
    masc[col] = masc[col].astype(str).str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    
#Creo la columna ganador

masc['ganador'] = -1

ganadorA1 = filtroGrandSlam & filtroGrandSlamGanaA
masc.loc[ganadorA1, 'ganador'] = 0

ganadorB1 = filtroGrandSlam & filtroGrandSlamGanaB
masc.loc[ganadorB1, 'ganador'] = 1

ganadorA2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaA
masc.loc[ganadorA2, 'ganador'] = 0

ganadorB2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaB
masc.loc[ganadorB2, 'ganador'] = 1

masc = masc[masc['acesA'] != '31.05.2025 00:16']
masc = masc.sort_values('fecha', ascending=False).reset_index(drop=True)


Ahora vamos a tratar los nan

In [160]:
filtro = masc.isna().sum() == 0
len(masc.isna().sum()[filtro]) #hay 37 columnas con 0 nan

37

In [161]:
filtro = (masc.isna().sum() > 0) & (masc.isna().sum() <= 900)
masc.isna().sum()[filtro] #las vairables cuotaA y cuotaB tienen 625 nan

cuotaA    625
cuotaB    625
dtype: int64

In [162]:
filtro = (masc.isna().sum() > 900) & (masc.isna().sum() <= 1000)
len(masc.isna().sum()[filtro]) #hay 30 columnas con 915 nan, eso puede significar que 915 datos sean incompletos
print(masc.isna().sum()[filtro].index.tolist())

['acesA', 'acesB', 'porcentajePrimerSaqueA', 'porcentajePrimerSaqueB', 'porcentajeBreakPointsSalvadosA', 'porcentajeBreakPointsSalvadosB', 'porcentajePuntosGanadosAlRestoConPrimerSaqueA', 'porcentajePuntosGanadosAlRestoConPrimerSaqueB', 'porcentajePuntosGanadosAlRestoConSegundoSaqueA', 'porcentajePuntosGanadosAlRestoConSegundoSaqueB', 'porcentajePuntosGanadosA', 'porcentajePuntosGanadosB', 'porcentajeJuegosGanadosA', 'porcentajeJuegosGanadosB', 'dobleA', 'dobleB', 'porcentajeJuegosGanadosPrimerSaqueA', 'porcentajeJuegosGanadosPrimerSaqueB', 'porcentajeJuegosGanadosSegundoSaqueA', 'porcentajeJuegosGanadosSegundoSaqueB', 'porcentajePuntosGanadosAlRestoA', 'porcentajePuntosGanadosAlRestoB', 'porcentajePuntosBreakConvertidosA', 'porcentajePuntosBreakConvertidosB', 'porcentajePuntosGanadosAlSaqueA', 'porcentajePuntosGanadosAlSaqueB', 'porcentajeJuegosGanadosAlSaqueA', 'porcentajeJuegosGanadosAlSaqueB', 'porcentajeJuegosGanadosAlRestoA', 'porcentajeJuegosGanadosAlRestoB']


In [163]:
filtro = (masc.isna().sum() > 1000) & (masc.isna().sum() <= 1700)
masc.isna().sum()[filtro] #4 variables con 1344 nan, las eliminamos
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

['PuntosGanadosUltimas10BolasA', 'PuntosGanadosUltimas10BolasB', 'BolasDePartidoSalvadasA', 'BolasDePartidoSalvadasB']


In [164]:
filtro = (masc.isna().sum() > 1700) & (masc.isna().sum() <= 2300)
masc.isna().sum()[filtro] #6 variables con 1800 nan, las eliminamos
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

['winnersA', 'winnersB', 'erroresNoForzadosA', 'erroresNoForzadosB', 'porcentajePuntosGanadosRedA', 'porcentajePuntosGanadosRedB']


In [165]:
filtro = (masc.isna().sum() > 2300)
len(masc.isna().sum()[filtro]) # hay 50 variables que casi todo es nan por lo que optamos por eliminarlas
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

['dobleFaltasA', 'dobleFaltasB', 'porcentajePuntosGanadosConPrimerSaqueA', 'porcentajePuntosGanadosConPrimerSaqueB', 'puntosGanadosConPrimerSaqueA', 'puntosGanadosConPrimerSaqueB', 'porcentajePuntosGanadosConSegundoSaqueA', 'porcentajePuntosGanadosConSegundoSaqueB', 'puntosGanadosConSegundoSaqueA', 'puntosGanadosConSegundoSaqueB', 'breakPointsSalvadosA', 'breakPointsSalvadosB', 'porcentajeBreakPointsA', 'porcentajeBreakPointsB', 'breakPointsA', 'breakPointsB', 'velocidadMediaPrimerSaqueA', 'velocidadMediaPrimerSaqueB', 'velocidadMediaSegundoSaqueA', 'velocidadMediaSegundoSaqueB', 'puntosGanadosAlRestoConPrimerSaqueA', 'puntosGanadosAlRestoConPrimerSaqueB', 'puntosGanadosAlRestoConSegundoSaqueA', 'puntosGanadosAlRestoConSegundoSaqueB', 'puntosGanadosRedA', 'puntosGanadosRedB', 'porcentajePuntosGanadosSaqueA', 'porcentajePuntosGanadosSaqueB', 'puntosGanadosSaqueA', 'puntosGanadosSaqueB', 'porcentajePuntosGanadosRestoA', 'porcentajePuntosGanadosRestoB', 'puntosGanadosRestoA', 'puntosGanad

Vemos como se ha quedado el df

In [166]:
masc.shape  #quedan 69 variables

(2567, 69)